# Net Position Analysis: From Net-to-Robot to Robot

This notebook analyzes the relationship between:
1. **Net position from robot's perspective** (sonar-detected distance and angle)
2. **Robot position relative to the net** (navigation-based positioning)

The analysis transforms between these two coordinate systems using CSV data from net detection results and navigation data.

## 1. Import Libraries and Utils

Import necessary libraries and our custom net position analysis utilities.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Import functional analysis API and config
from utils.config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS, IMAGE_PROCESSING_CONFIG, TRACKING_CONFIG

# CONFIG VERIFICATION - Print exact values to ensure consistency
print("=== IMAGE_PROCESSING_CONFIG VALUES ===")
for key, value in IMAGE_PROCESSING_CONFIG.items():
    print(f"{key}: {value}")

print("\n=== TRACKING_CONFIG VALUES ===")
for key, value in TRACKING_CONFIG.items():
    print(f"{key}: {value}")

print("\nLibraries and utilities imported successfully!")

=== IMAGE_PROCESSING_CONFIG VALUES ===
binary_threshold: 1
use_advanced_momentum_merging: True
adaptive_angle_steps: 20
adaptive_base_radius: 3
adaptive_max_elongation: 1.0
momentum_boost: 10.0
adaptive_linearity_threshold: 1.0
downscale_factor: 2
top_k_bins: 8
min_coverage_percent: 0.3
gaussian_sigma: 5.0
basic_gaussian_kernel_size: 3
basic_gaussian_sigma: 5.0
basic_momentum_boost: 5.0
morph_close_kernel: 0
edge_dilation_iterations: 0
min_contour_area: 200
aoi_boost_factor: 10.0
max_distance_change_pixels: 20

=== TRACKING_CONFIG VALUES ===
use_elliptical_aoi: True
ellipse_expansion_factor: 0.5
center_smoothing_alpha: 0.3
ellipse_size_smoothing_alpha: 0.01
ellipse_orientation_smoothing_alpha: 0.2
ellipse_max_movement_pixels: 30.0
use_corridor_splitting: True
corridor_band_k: 2.0
corridor_length_px: None
corridor_length_factor: 2.0
corridor_widen: 1.0
corridor_both_directions: True
max_frames_without_detection: 30
aoi_decay_factor: 0.98

Libraries and utilities imported successfully!


## 2. Configuration and Setup

Set your target bag and load data paths.

In [12]:
# Configuration - set your target bag name here
TARGET_BAG = "2024-08-20_17-02-00"  # Change this to your desired bag ID
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

print(f"Target Bag: {TARGET_BAG}")
print(f"Exports Folder: {EXPORTS_FOLDER}")
print(f"By-Bag Folder: {BY_BAG_FOLDER}")

# NOTE: To export data for this specific bag, use:
# python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem 2024-08-20_17-02-00 --all

Target Bag: 2024-08-20_17-02-00
Exports Folder: /Volumes/LaCie/SOLAQUA/exports
By-Bag Folder: /Volumes/LaCie/SOLAQUA/exports/by_bag


## 3. Load Sonar Analysis Results

Load the sonar distance/angle detections from the analysis CSV file.

In [21]:
# Load sonar distance analysis results
print("Loading sonar analysis results...\n")

output_dir = EXPORTS_FOLDER / EXPORTS_SUBDIRS['outputs']

# Try multiple possible filenames
possible_filenames = [
    f"{TARGET_BAG}_analysis.csv",
    f"{TARGET_BAG}_data_cones_analysis.csv",
    f"{TARGET_BAG}_video_cones_analysis.csv",
]

analysis_file = None
for filename in possible_filenames:
    candidate = output_dir / filename
    if candidate.exists():
        analysis_file = candidate
        break

if analysis_file is None:
    print(f"✗ Sonar analysis file not found\n")
    print(f"Looked for:")
    for filename in possible_filenames:
        print(f"  - {output_dir / filename}")
    print(f"\n⚠️  FILE NOT YET GENERATED\n")
    print(f"To generate sonar analysis results:")
    print(f"1. Run the PIPELINE VIDEO cell in notebook 04")
    print(f"2. Then run the ANALYZING DISTANCE OVER TIME cell")
    print(f"   - This will process all 1500 frames")
    print(f"   - Takes several minutes")
    print(f"   - Saves results to outputs folder\n")
    print(f"✓ Analysis complete! You should now see the file.\n")
    df_sonar = None
elif analysis_file.exists():
    df_sonar = pd.read_csv(analysis_file)
    df_sonar['timestamp'] = pd.to_datetime(df_sonar['timestamp'])
    df_sonar = df_sonar.sort_values('timestamp')
    
    print(f"✓ Loaded sonar analysis: {len(df_sonar)} frames")
    print(f"From: {analysis_file.name}")
    print(f"Columns: {list(df_sonar.columns)}")
    print(f"Detection success rate: {df_sonar['detection_success'].mean()*100:.1f}%")
    print(f"\nSample data:")
    print(df_sonar[['timestamp', 'distance_meters', 'angle_degrees', 'detection_success']].head())
else:
    df_sonar = None

Loading sonar analysis results...

✓ Loaded sonar analysis: 944 frames
From: 2024-08-20_17-02-00_data_cones_analysis.csv
Columns: ['frame_index', 'timestamp', 'distance_pixels', 'distance_meters', 'angle_degrees', 'detection_success', 'tracking_status', 'area']
Detection success rate: 100.0%

Sample data:
                            timestamp  distance_meters  angle_degrees  \
0 2024-08-20 15:02:03.319382668+00:00         0.565697     171.300026   
1 2024-08-20 15:02:03.405480623+00:00         0.566433     171.296124   
2 2024-08-20 15:02:03.458926201+00:00         0.562703     171.240139   
3 2024-08-20 15:02:03.505110979+00:00         0.563517     171.184953   
4 2024-08-20 15:02:03.609708309+00:00         0.557811     171.306121   

   detection_success  
0               True  
1               True  
2               True  
3               True  
4               True  


In [22]:
# Load navigation data (DVL)
print("Loading DVL navigation data...")

nav_file = BY_BAG_FOLDER / f"navigation_plane_approximation__{TARGET_BAG}_data.csv"

if nav_file.exists():
    print(f"✓ Found navigation file: {nav_file.name}")
    df_nav = pd.read_csv(nav_file)
    
    # Apply same transformations as video generation code
    df_nav["timestamp"] = pd.to_datetime(df_nav["ts_utc"])
    df_nav = df_nav.sort_values("timestamp")
    
    print(f"✓ Loaded navigation data: {len(df_nav)} records")
    print(f"Columns: {list(df_nav.columns)}")
    
    # Check for the key columns that video generation uses
    key_columns = [c for c in ["NetDistance", "NetPitch", "timestamp"] if c in df_nav.columns]
    print(f"Key DVL columns available: {key_columns}")
    
    # Display sample data focusing on the columns used by video generation
    if 'NetDistance' in df_nav.columns:
        valid_distances = df_nav['NetDistance'].dropna()
        print(f"\nNetDistance: {len(valid_distances)} valid measurements")
        print(f"  Range: {valid_distances.min():.3f} to {valid_distances.max():.3f} m")
        print(f"  Mean: {valid_distances.mean():.3f} ± {valid_distances.std():.3f} m")
    
    if 'NetPitch' in df_nav.columns:
        valid_pitch = df_nav['NetPitch'].dropna()
        print(f"NetPitch: {len(valid_pitch)} valid measurements")
        print(f"  Range: {np.degrees(valid_pitch.min()):.1f} to {np.degrees(valid_pitch.max()):.1f}°")
        print(f"  Mean: {np.degrees(valid_pitch.mean()):.1f} ± {np.degrees(valid_pitch.std()):.1f}°")
    
    print(f"\nSample navigation data (first 3 rows):")
    display_cols = [c for c in ['timestamp', 'NetDistance', 'NetPitch'] if c in df_nav.columns]
    print(df_nav[display_cols].head(3))
    
else:
    print(f"✗ Navigation file not found: {nav_file}")
    
    # Fallback: Try using distance_measurement module
    try:
        import utils.distance_measurement as sda
        raw_data, _ = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)
        
        if raw_data and 'navigation' in raw_data and raw_data['navigation'] is not None:
            df_nav = raw_data['navigation'].copy()
            df_nav['timestamp'] = pd.to_datetime(df_nav['timestamp'], errors='coerce')
            print(f"✓ Loaded via fallback method: {len(df_nav)} records")
        else:
            df_nav = None
            print("✗ No navigation data found")
    except Exception as e:
        print(f"✗ Failed to load navigation data: {e}")
        df_nav = None

Loading DVL navigation data...
✓ Found navigation file: navigation_plane_approximation__2024-08-20_17-02-00_data.csv
✓ Loaded navigation data: 497 records
Columns: ['t', 't_header', 't_bag', 't_src', 'bag', 'bag_file', 'topic', 'NormalDVL', 'Altitude', 'NetDistance', 'NetHeading', 'NetPitch', 'NetLock', 'NetVelocity_u', 'NetVelocity_v', 'NetVelocity_w', '__msgtype__', 't0', 't_rel', 'ts_utc', 'ts_oslo', 'timestamp']
Key DVL columns available: ['NetDistance', 'NetPitch', 'timestamp']

NetDistance: 497 valid measurements
  Range: 0.430 to 3.340 m
  Mean: 1.121 ± 0.305 m
NetPitch: 497 valid measurements
  Range: -60.5 to 85.3°
  Mean: -1.8 ± 13.0°

Sample navigation data (first 3 rows):
                            timestamp  NetDistance  NetPitch
0 2024-08-20 15:02:05.408126593+00:00         0.72  -0.07037
1 2024-08-20 15:02:05.503421782+00:00         0.76  -0.09843
2 2024-08-20 15:02:05.603994131+00:00         0.77  -0.06585


In [23]:
# Verify both datasets are loaded and compare
if df_sonar is not None and df_nav is not None:
    print("✓ Both sonar and navigation datasets loaded successfully\n")
    
    print("=== SONAR ANALYSIS SUMMARY ===")
    print(f"Total frames: {len(df_sonar)}")
    print(f"Successful detections: {df_sonar['detection_success'].sum()}")
    print(f"Detection rate: {df_sonar['detection_success'].mean()*100:.1f}%")
    
    if df_sonar['distance_meters'].notna().sum() > 0:
        valid_distances = df_sonar['distance_meters'].dropna()
        print(f"\nDistance measurements:")
        print(f"  Valid: {len(valid_distances)}")
        print(f"  Range: {valid_distances.min():.3f} - {valid_distances.max():.3f} m")
        print(f"  Mean: {valid_distances.mean():.3f} ± {valid_distances.std():.3f} m")
    
    if df_sonar['angle_degrees'].notna().sum() > 0:
        valid_angles = df_sonar['angle_degrees'].dropna()
        print(f"\nAngle measurements:")
        print(f"  Valid: {len(valid_angles)}")
        print(f"  Range: {valid_angles.min():.1f} - {valid_angles.max():.1f}°")
        print(f"  Mean: {valid_angles.mean():.1f} ± {valid_angles.std():.1f}°")
    
    print("\n=== DVL NAVIGATION SUMMARY ===")
    print(f"Total records: {len(df_nav)}")
    
    if 'NetDistance' in df_nav.columns and df_nav['NetDistance'].notna().sum() > 0:
        valid_nav_dist = df_nav['NetDistance'].dropna()
        print(f"\nNetDistance measurements:")
        print(f"  Valid: {len(valid_nav_dist)}")
        print(f"  Range: {valid_nav_dist.min():.3f} - {valid_nav_dist.max():.3f} m")
        print(f"  Mean: {valid_nav_dist.mean():.3f} ± {valid_nav_dist.std():.3f} m")
    
    if 'NetPitch' in df_nav.columns and df_nav['NetPitch'].notna().sum() > 0:
        valid_nav_pitch = df_nav['NetPitch'].dropna()
        print(f"\nNetPitch measurements:")
        print(f"  Valid: {len(valid_nav_pitch)}")
        print(f"  Range: {np.degrees(valid_nav_pitch.min()):.1f} - {np.degrees(valid_nav_pitch.max()):.1f}°")
        print(f"  Mean: {np.degrees(valid_nav_pitch.mean()):.1f} ± {np.degrees(valid_nav_pitch.std()):.1f}°")
    
    print("\n✓ Ready for analysis")
    
else:
    print("✗ Cannot proceed - missing sonar or navigation data")
    print("Available:")
    print(f"  Sonar: {'✓' if df_sonar is not None else '✗'}")
    print(f"  Navigation: {'✓' if df_nav is not None else '✗'}")

✓ Both sonar and navigation datasets loaded successfully

=== SONAR ANALYSIS SUMMARY ===
Total frames: 944
Successful detections: 944
Detection rate: 100.0%

Distance measurements:
  Valid: 944
  Range: 0.557 - 4.596 m
  Mean: 1.265 ± 0.792 m

Angle measurements:
  Valid: 944
  Range: 118.6 - 197.5°
  Mean: 170.7 ± 14.6°

=== DVL NAVIGATION SUMMARY ===
Total records: 497

NetDistance measurements:
  Valid: 497
  Range: 0.430 - 3.340 m
  Mean: 1.121 ± 0.305 m

NetPitch measurements:
  Valid: 497
  Range: -60.5 - 85.3°
  Mean: -1.8 ± 13.0°

✓ Ready for analysis


## 4. Synchronize and Compare Systems

Synchronize sonar and DVL measurements and calculate statistics.

In [24]:
# Synchronize sonar and DVL data
if df_sonar is not None and df_nav is not None:
    print("=== SYNCHRONIZING SONAR AND DVL DATA ===\n")
    
    # Synchronize timestamps
    # Use sonar timestamps as reference
    sync_data = []
    
    for idx, sonar_row in df_sonar.iterrows():
        sonar_ts = sonar_row['timestamp']
        
        # Find closest DVL measurement
        if 'NetDistance' in df_nav.columns and 'timestamp' in df_nav.columns:
            time_diffs = abs(df_nav['timestamp'] - sonar_ts)
            closest_nav_idx = time_diffs.idxmin()
            closest_nav = df_nav.loc[closest_nav_idx]
            time_diff = time_diffs.iloc[closest_nav_idx].total_seconds()
            
            # Only use if within tolerance (1 second)
            if time_diff <= 1.0:
                sync_row = {
                    'timestamp': sonar_ts,
                    'sonar_distance_m': sonar_row.get('distance_meters'),
                    'sonar_angle_deg': sonar_row.get('angle_degrees'),
                    'sonar_detection': sonar_row.get('detection_success', False),
                    'dvl_distance_m': closest_nav.get('NetDistance'),
                    'dvl_pitch_deg': np.degrees(closest_nav.get('NetPitch')) if 'NetPitch' in closest_nav and pd.notna(closest_nav.get('NetPitch')) else None,
                    'time_diff_s': time_diff
                }
                sync_data.append(sync_row)
    
    sync_df = pd.DataFrame(sync_data)
    
    print(f"✓ Synchronized {len(sync_df)} sonar-DVL measurement pairs")
    print(f"\nSynchronized data sample:")
    print(sync_df[['timestamp', 'sonar_distance_m', 'dvl_distance_m', 'sonar_angle_deg', 'dvl_pitch_deg']].head(10))
    
    # Calculate comparison statistics
    valid_sonar = sync_df['sonar_distance_m'].dropna()
    valid_dvl = sync_df['dvl_distance_m'].dropna()
    
    if len(valid_sonar) > 0 and len(valid_dvl) > 0:
        distance_diff = valid_sonar - valid_dvl
        
        print(f"\n=== DISTANCE COMPARISON STATISTICS ===")
        print(f"Sonar - DVL distance difference:")
        print(f"  Mean: {distance_diff.mean():.3f} m")
        print(f"  Std: {distance_diff.std():.3f} m")
        print(f"  Min: {distance_diff.min():.3f} m")
        print(f"  Max: {distance_diff.max():.3f} m")
        print(f"  95th percentile: {np.percentile(distance_diff, 95):.3f} m")
        
        # Calculate correlation if enough data
        if len(valid_sonar) > 10:
            correlation = valid_sonar.corr(valid_dvl)
            print(f"  Correlation: {correlation:.3f}")
    
else:
    print("✗ Cannot synchronize - missing data")

=== SYNCHRONIZING SONAR AND DVL DATA ===

✓ Synchronized 927 sonar-DVL measurement pairs

Synchronized data sample:
                            timestamp  sonar_distance_m  dvl_distance_m  \
0 2024-08-20 15:02:04.418097496+00:00          0.565798            0.72   
1 2024-08-20 15:02:04.460487843+00:00          0.566812            0.72   
2 2024-08-20 15:02:04.520919323+00:00          0.569042            0.72   
3 2024-08-20 15:02:04.586270809+00:00          0.569081            0.72   
4 2024-08-20 15:02:04.647171021+00:00          0.570139            0.72   
5 2024-08-20 15:02:04.712663174+00:00          0.569644            0.72   
6 2024-08-20 15:02:04.801765203+00:00          0.569205            0.72   
7 2024-08-20 15:02:04.849609375+00:00          0.570380            0.72   
8 2024-08-20 15:02:04.905240774+00:00          0.567905            0.72   
9 2024-08-20 15:02:04.978989840+00:00          0.567068            0.72   

   sonar_angle_deg  dvl_pitch_deg  
0       169.714729    

## 5. Load FFT Data (Optional)

Load relative FFT pose data if available for three-system comparison.

In [25]:
# Load relative FFT data
print("=== LOADING RELATIVE FFT DATA ===\n")

FFT_CSV_PATH = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose") / f"{TARGET_BAG}_relative_pose_fft.csv"

print(f"FFT CSV Path: {FFT_CSV_PATH}")
print(f"File exists: {FFT_CSV_PATH.exists()}\n")

df_fft = None
if FFT_CSV_PATH.exists():
    try:
        df_fft = pd.read_csv(FFT_CSV_PATH)
        
        # Ensure timestamp column
        if 'time' in df_fft.columns:
            df_fft['timestamp'] = pd.to_numeric(df_fft['time'], errors='coerce')
        
        print(f"✓ Loaded FFT data: {len(df_fft)} records")
        print(f"Columns: {list(df_fft.columns)}")
        
        if 'distance' in df_fft.columns:
            valid_dist = df_fft['distance'].dropna()
            print(f"\nDistance: {len(valid_dist)} valid measurements")
            print(f"  Range: {valid_dist.min():.3f} - {valid_dist.max():.3f} m")
            print(f"  Mean: {valid_dist.mean():.3f} ± {valid_dist.std():.3f} m")
        
        if 'pitch' in df_fft.columns:
            valid_pitch = df_fft['pitch'].dropna()
            print(f"\nPitch: {len(valid_pitch)} valid measurements")
            print(f"  Range: {np.degrees(valid_pitch.min()):.1f} - {np.degrees(valid_pitch.max()):.1f}°")
            print(f"  Mean: {np.degrees(valid_pitch.mean()):.1f} ± {np.degrees(valid_pitch.std()):.1f}°")
        
        print(f"\nSample FFT data:")
        print(df_fft[['timestamp', 'distance', 'pitch']].head() if 'pitch' in df_fft.columns else df_fft[['timestamp', 'distance']].head())
        
    except Exception as e:
        print(f"✗ Error loading FFT data: {e}")
        df_fft = None
else:
    print(f"✗ FFT data file not found")
    print(f"  Continuing with two-system comparison (Sonar + DVL)\n")

=== LOADING RELATIVE FFT DATA ===

FFT CSV Path: /Volumes/LaCie/SOLAQUA/relative_fft_pose/2024-08-20_17-02-00_relative_pose_fft.csv
File exists: True

✓ Loaded FFT data: 410 records
Columns: ['time', 'distance', 'heading', 'pitch', 'timestamp']

Distance: 410 valid measurements
  Range: -183.272 - 159.623 m
  Mean: 87.669 ± 31.986 m

Pitch: 410 valid measurements
  Range: -82.7 - 88.1°
  Mean: 3.8 ± 14.6°

Sample FFT data:
      timestamp  distance     pitch
0  1.724166e+09  0.000323  0.750176
1  1.724166e+09  0.000087  0.586265
2  1.724166e+09  0.024386  1.438593
3  1.724166e+09  0.093198  1.428290
4  1.724166e+09  0.000892  0.817889


## 6. Distance and Pitch Comparison

Compare distance and pitch measurements across sonar, DVL, and FFT systems.

In [28]:
# Distance and Pitch Comparison
if df_sonar is not None and df_nav is not None and df_fft is not None:
    print("=== DISTANCE AND PITCH COMPARISON ===\n")
    
    # Import visualization utilities
    from utils.multi_system_sync import (
        NetRelativePositionCalculator,
        MultiSystemSynchronizer,
        NetRelativeVisualizer
    )
    
    # Initialize tools
    calculator = NetRelativePositionCalculator()
    synchronizer = MultiSystemSynchronizer(tolerance_seconds=0.5)
    visualizer = NetRelativeVisualizer()
    
    # Calculate net positions for each system
    print("Calculating net relative positions...")
    df_sonar_pos = calculator.calculate_sonar_net_position(df_sonar, apply_angle_correction=True)
    df_nav_pos = calculator.calculate_dvl_net_position(df_nav)
    
    # Process FFT data
    df_fft_proc = df_fft.copy()
    if 'time' in df_fft_proc.columns:
        df_fft_proc['timestamp'] = pd.to_datetime(df_fft_proc['time'], unit='s')
    
    # Handle distance unit conversion
    if 'distance' in df_fft_proc.columns:
        distances = pd.to_numeric(df_fft_proc['distance'], errors='coerce')
        max_dist = distances.abs().max()
        
        if max_dist > 50:  # Likely cm
            df_fft_proc['distance_m'] = distances / 100.0
            print(f"Converted FFT distances from cm to m (max: {max_dist:.1f}cm → {max_dist/100:.2f}m)")
        else:
            df_fft_proc['distance_m'] = distances
            print(f"FFT distances already in meters (max: {max_dist:.3f}m)")
    
    # Convert pitch to degrees and calculate positions
    if 'pitch' in df_fft_proc.columns:
        df_fft_proc['pitch_rad'] = pd.to_numeric(df_fft_proc['pitch'], errors='coerce')
        df_fft_proc['pitch_deg'] = df_fft_proc['pitch_rad'] * 180 / np.pi
        df_fft_proc['fft_x_m'] = df_fft_proc['distance_m'] * np.cos(df_fft_proc['pitch_rad'])
        df_fft_proc['fft_y_m'] = df_fft_proc['distance_m'] * np.sin(df_fft_proc['pitch_rad'])
    
    df_fft_pos = df_fft_proc
    
    # Synchronize systems
    print("Synchronizing three systems...")
    sync_df = synchronizer.synchronize_three_systems(df_fft_pos, df_sonar_pos, df_nav_pos)
    print(f"✓ Synchronized {len(sync_df)} time points\n")
    
    # Create and display distance comparison
    print("Creating distance comparison plot...")
    fig_distance = visualizer.create_distance_comparison(sync_df, TARGET_BAG)
    fig_distance.show()
    
    # Create and display pitch comparison
    print("Creating pitch comparison plot...")
    fig_pitch = visualizer.create_pitch_comparison(sync_df, TARGET_BAG)
    fig_pitch.show()
    
    print("\n✓ Distance and pitch comparison complete")

else:
    print("✗ Cannot compare - missing sonar, DVL, or FFT data")

=== DISTANCE AND PITCH COMPARISON ===

Calculating net relative positions...
Converted FFT distances from cm to m (max: 183.3cm → 1.83m)
Synchronizing three systems...
✓ Synchronized 1851 time points

Creating distance comparison plot...


Creating pitch comparison plot...



✓ Distance and pitch comparison complete


## 7. Net Relative Position (X,Y) Comparison

Compare net relative X and Y positions calculated from all three systems using distance and pitch/angle measurements.

In [29]:
# Net Relative X,Y Position Comparison
if 'sync_df' in locals() and sync_df is not None:
    print("=== NET RELATIVE POSITION (X,Y) COMPARISON ===\n")
    
    # Display position summary
    print("Position Summary Statistics:")
    visualizer.print_position_summary(sync_df)
    
    # Create and display X position comparison
    print("\nCreating X position comparison plot...")
    fig_x_position = visualizer.create_x_position_comparison(sync_df, TARGET_BAG)
    fig_x_position.show()
    
    # Create and display Y position comparison
    print("\nCreating Y position comparison plot...")
    fig_y_position = visualizer.create_y_position_comparison(sync_df, TARGET_BAG)
    fig_y_position.show()
    
    # Calculate position statistics
    print("\n=== POSITION STATISTICS ===")
    
    for system, x_col, y_col in [
        ('FFT', 'fft_x_m', 'fft_y_m'),
        ('Sonar', 'sonar_x_m', 'sonar_y_m'),
        ('DVL', 'nav_x_m', 'nav_y_m')
    ]:
        if x_col in sync_df.columns and y_col in sync_df.columns:
            valid = sync_df[x_col].notna() & sync_df[y_col].notna()
            if valid.sum() > 0:
                x_data = sync_df.loc[valid, x_col]
                y_data = sync_df.loc[valid, y_col]
                
                # Calculate distance from origin
                dist_from_origin = np.sqrt(x_data**2 + y_data**2)
                
                print(f"\n{system}:")
                print(f"  X: {x_data.mean():.3f} ± {x_data.std():.3f} m (range: {x_data.min():.3f} to {x_data.max():.3f})")
                print(f"  Y: {y_data.mean():.3f} ± {y_data.std():.3f} m (range: {y_data.min():.3f} to {y_data.max():.3f})")
                print(f"  Distance from origin: {dist_from_origin.mean():.3f} ± {dist_from_origin.std():.3f} m")
    
    print("\n✓ Net relative position comparison complete")

else:
    print("✗ Synchronization data not available - run distance/pitch comparison first")

=== NET RELATIVE POSITION (X,Y) COMPARISON ===

Position Summary Statistics:
Position data summary:
  FFT: 1769 points, X: 0.82±0.38, Y: 0.04±0.12
  Sonar: 1851 points, X: 1.08±0.32, Y: -0.23±0.65
  DVL: 1813 points, X: 1.10±0.29, Y: -0.08±0.28

Creating X position comparison plot...



Creating Y position comparison plot...



=== POSITION STATISTICS ===

FFT:
  X: 0.820 ± 0.379 m (range: -1.821 to 1.573)
  Y: 0.039 ± 0.122 m (range: -0.596 to 0.441)
  Distance from origin: 0.849 ± 0.335 m

Sonar:
  X: 1.077 ± 0.323 m (range: 0.550 to 2.153)
  Y: -0.231 ± 0.651 m (range: -3.923 to 0.473)
  Distance from origin: 1.163 ± 0.623 m

DVL:
  X: 1.097 ± 0.293 m (range: 0.046 to 2.893)
  Y: -0.080 ± 0.285 m (range: -1.755 to 0.786)
  Distance from origin: 1.126 ± 0.332 m

✓ Net relative position comparison complete
